In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pyarrow as pa
import pyarrow.parquet as pq
import geopandas as gpd
import seaborn as sns

#Packages
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import numpy as np
from scipy import stats
pd.set_option('display.max_columns', None)

from scipy.stats import skew, kurtosis

import pygris
import shapely

In [ ]:
df = pd.read_parquet("FimaNfipClaims.parquet.gzip")

In [ ]:
df_copy = df[['state', 'reportedZipCode', 'countyCode', 'censusTract', 'censusBlockGroupFips', 'latitude', 'longitude', 'yearOfLoss']].drop_duplicates()

## Using Pygris

In [ ]:
states = pygris.states()
states_list = sorted(states['STUSPS'].unique())

## BlockGroup Shapefile

In [ ]:
BG_list_2022 = []

for i in states_list:
    BG_list_2022.append(pygris.block_groups(state = i, year = 2022))

In [ ]:
BG_list_2020 = []

for i in states_list:
    BG_list_2020.append(pygris.block_groups(state = i, year = 2020))

In [ ]:
BG_list_2019 = []

for i in states_list:
    BG_list_2019.append(pygris.block_groups(state = i, year = 2019))

In [ ]:
BG_list_2018 = []

for i in states_list:
    BG_list_2018.append(pygris.block_groups(state = i, year = 2018))

In [ ]:
BG_list_2017 = []

for i in states_list:
    BG_list_2017.append(pygris.block_groups(state = i, year = 2017))

In [ ]:
BG_list_2016 = []

for i in states_list:
    BG_list_2016.append(pygris.block_groups(state = i, year = 2016))

In [ ]:
BG_list_2015 = []

for i in states_list:
    BG_list_2015.append(pygris.block_groups(state = i, year = 2015))

In [ ]:
BG_list_2014 = []

for i in states_list:
    BG_list_2014.append(pygris.block_groups(state = i, year = 2014))

In [ ]:
BG_list_2013 = []

for i in states_list:
    BG_list_2013.append(pygris.block_groups(state = i, year = 2013))

In [ ]:
BG_list_2012 = []

for i in states_list:
    BG_list_2012.append(pygris.block_groups(state = i, year = 2012))

In [ ]:
BG_list_2011 = []

for i in states_list:
    BG_list_2011.append(pygris.block_groups(state = i, year = 2011))

In [ ]:
BG_list_2010 = []

for i in states_list:
    BG_list_2010.append(pygris.block_groups(state = i, year = 2010))

In [ ]:
BG_list_2000 = []

for i in states_list:
    BG_list_2000.append(pygris.block_groups(state = i, year = 2000))

In [ ]:
BG_list_1990 = []

for i in states_list:
    BG_list_1990.append(pygris.block_groups(state = i, year = 1990, cb = True))

In [ ]:
temp2022 = pd.concat(BG_list_2022, ignore_index=True)
temp2020 = pd.concat(BG_list_2020, ignore_index=True)
temp2019 = pd.concat(BG_list_2019, ignore_index=True)
temp2018 = pd.concat(BG_list_2018, ignore_index=True)
temp2017 = pd.concat(BG_list_2017, ignore_index=True)
temp2016 = pd.concat(BG_list_2016, ignore_index=True)
temp2015 = pd.concat(BG_list_2015, ignore_index=True)
temp2014 = pd.concat(BG_list_2014, ignore_index=True)
temp2013 = pd.concat(BG_list_2013, ignore_index=True)
temp2012 = pd.concat(BG_list_2012, ignore_index=True)
temp2011 = pd.concat(BG_list_2011, ignore_index=True)
temp2010 = pd.concat(BG_list_2010, ignore_index=True)
temp2000 = pd.concat(BG_list_2000, ignore_index=True)
temp1990 = pd.concat(BG_list_1990, ignore_index=True)

In [ ]:
temp2022 = temp2022[['GEOID', 'geometry']]
temp2022['year'] = 2020

temp2020 = temp2020[['GEOID', 'geometry']]
temp2020['year'] = 2020

temp2019 = temp2019[['GEOID', 'geometry']]
temp2019['year'] = 2010

temp2018 = temp2018[['GEOID', 'geometry']]
temp2018['year'] = 2010

temp2017 = temp2017[['GEOID', 'geometry']]
temp2017['year'] = 2010

temp2016 = temp2016[['GEOID', 'geometry']]
temp2016['year'] = 2010

temp2015 = temp2015[['GEOID', 'geometry']]
temp2015['year'] = 2010

temp2014 = temp2014[['GEOID', 'geometry']]
temp2014['year'] = 2010

temp2013 = temp2013[['GEOID', 'geometry']]
temp2013['year'] = 2010

temp2012 = temp2012[['GEOID', 'geometry']]
temp2012['year'] = 2010

temp2011 = temp2011[['GEOID', 'geometry']]
temp2011['year'] = 2010

temp2010 = temp2010[['GEOID10', 'geometry']]
temp2010['year'] = 2010

temp2010 = temp2010.rename(columns={"GEOID10": "GEOID"})

temp2000 = temp2000[['BKGPIDFP00', 'geometry']]
temp2000['year'] = 2000

temp2000 = temp2000.rename(columns={"BKGPIDFP00": "GEOID"})

temp1990 = temp1990[['GEOID', 'geometry']]
temp1990['year'] = 1990

In [ ]:
list_censusBG = temp2022['GEOID'].dropna().drop_duplicates().astype(str).tolist()

# add preceding 0 to make 4-digit zip codes into 5-digit
list_censusBG = [censusBG.zfill(12) for censusBG in list_censusBG]

In [ ]:
len(list_censusBG)

In [ ]:
list_censusBG_2 = list_censusBG_2 + list_censusBG

In [ ]:
len(list_censusBG_2)

In [ ]:
list_censusBG_2 = list(set(list_censusBG_2))

In [ ]:
len(list_censusBG_2)

In [ ]:
#CensusBG list for NC

list_censusBG = df['censusBlockGroupFips'].dropna().drop_duplicates().tolist()

list_censusBG = [str(int(float(i))) for i in list_censusBG]

# add preceding 0 to make 4-digit zip codes into 5-digit
list_censusBG = [censusBG.zfill(12) for censusBG in list_censusBG]

In [ ]:
# Convert both lists to sets
set_bg = set(list_censusBG)
set_bg_2 = set(list_censusBG_2)

# Find the intersection of the two sets
common_bg = set_bg.intersection(set_bg_2)

# Print the number of common zip codes
print("Number of common BG:", len(common_bg))

In [ ]:
len(common_bg)/len(list_censusBG)

In [ ]:
len(common_bg)

In [ ]:
BG_list_df2022 = temp2022[temp2022['GEOID'].isin(common_bg)]
BG_list_df2020 = temp2020[temp2020['GEOID'].isin(common_bg)]
BG_list_df2019 = temp2019[temp2019['GEOID'].isin(common_bg)]
BG_list_df2018 = temp2018[temp2018['GEOID'].isin(common_bg)]
BG_list_df2017 = temp2017[temp2017['GEOID'].isin(common_bg)]
BG_list_df2016 = temp2016[temp2016['GEOID'].isin(common_bg)]
BG_list_df2015 = temp2015[temp2015['GEOID'].isin(common_bg)]
BG_list_df2014 = temp2014[temp2014['GEOID'].isin(common_bg)]
BG_list_df2013 = temp2013[temp2013['GEOID'].isin(common_bg)]
BG_list_df2012 = temp2012[temp2012['GEOID'].isin(common_bg)]
BG_list_df2011 = temp2011[temp2011['GEOID'].isin(common_bg)]
BG_list_df2010 = temp2010[temp2010['GEOID'].isin(common_bg)]
BG_list_df2000 = temp2000[temp2000['GEOID'].isin(common_bg)]
BG_list_df1990 = temp1990[temp1990['GEOID'].isin(common_bg)]

In [ ]:
BG_df = pd.concat([BG_list_df2022, BG_list_df2020, BG_list_df2019,BG_list_df2013,BG_list_df2012, BG_list_df2011, BG_list_df2010, BG_list_df2000, BG_list_df1990], ignore_index=True)

In [ ]:
BG_df2 = BG_df.drop_duplicates()

In [ ]:
BG_df.shape[0] 

In [ ]:
BG_df2.shape[0]

In [ ]:
BG_df2 = BG_df2.drop_duplicates(subset=['GEOID', 'year'])

In [ ]:
BG_df2.shape[0]

In [ ]:
chunk_size = 40000  # adjust based on your system's capabilities
chunks = [x for x in range(0, len(BG_df2), chunk_size)]

for start in chunks:
    end = start + chunk_size
    temp_df = BG_df2.iloc[start:end].copy()
    temp_df['geometry'] = temp_df['geometry'].apply(lambda geom: geom.wkt)
    temp_df.to_parquet(f"BG_geometry_{start}_{end}.parquet.gzip", compression='gzip')

## Census Tract Shapefile

In [ ]:
Tracts_list_1990 = []

for i in states_list:
    Tracts_list_1990.append(pygris.tracts(state = i, year = 1990, cb = True))

In [ ]:
Tracts_list_2000 = []

for i in states_list:
    Tracts_list_2000.append(pygris.tracts(state = i, year = 2000))

In [ ]:
Tracts_list_2010 = []

for i in states_list:
    Tracts_list_2010.append(pygris.tracts(state = i, year = 2010))

In [ ]:
Tracts_list_2011 = []

for i in states_list:
    Tracts_list_2011.append(pygris.tracts(state = i, year = 2011))

In [ ]:
Tracts_list_2012 = []

for i in states_list:
    Tracts_list_2012.append(pygris.tracts(state = i, year = 2012))

In [ ]:
Tracts_list_2013 = []

for i in states_list:
    Tracts_list_2013.append(pygris.tracts(state = i, year = 2013))

In [ ]:
Tracts_list_2014 = []

for i in states_list:
    Tracts_list_2014.append(pygris.tracts(state = i, year = 2014))

In [ ]:
Tracts_list_2015 = []

for i in states_list:
    Tracts_list_2015.append(pygris.tracts(state = i, year = 2015))

In [ ]:
Tracts_list_2016 = []

for i in states_list:
    Tracts_list_2016.append(pygris.tracts(state = i, year = 2016))

In [ ]:
Tracts_list_2017 = []

for i in states_list:
    Tracts_list_2017.append(pygris.tracts(state = i, year = 2017))

In [ ]:
Tracts_list_2018 = []

for i in states_list:
    Tracts_list_2018.append(pygris.tracts(state = i, year = 2018))

In [ ]:
Tracts_list_2019 = []

for i in states_list:
    Tracts_list_2019.append(pygris.tracts(state = i, year = 2019))

In [ ]:
Tracts_list_2020 = []

for i in states_list:
    Tracts_list_2020.append(pygris.tracts(state = i, year = 2020))

In [ ]:
Tracts_list_2022 = []

for i in states_list:
    Tracts_list_2022.append(pygris.tracts(state = i, year = 2022))

In [ ]:
temp2022 = pd.concat(Tracts_list_2022, ignore_index=True)
temp2020 = pd.concat(Tracts_list_2020, ignore_index=True)
temp2019 = pd.concat(Tracts_list_2019, ignore_index=True)
temp2018 = pd.concat(Tracts_list_2018, ignore_index=True)
temp2017 = pd.concat(Tracts_list_2017, ignore_index=True)
temp2016 = pd.concat(Tracts_list_2016, ignore_index=True)
temp2015 = pd.concat(Tracts_list_2015, ignore_index=True)
temp2014 = pd.concat(Tracts_list_2014, ignore_index=True)
temp2013 = pd.concat(Tracts_list_2013, ignore_index=True)
temp2012 = pd.concat(Tracts_list_2012, ignore_index=True)
temp2011 = pd.concat(Tracts_list_2011, ignore_index=True)
temp2010 = pd.concat(Tracts_list_2010, ignore_index=True)
temp2000 = pd.concat(Tracts_list_2000, ignore_index=True)
temp1990 = pd.concat(Tracts_list_1990, ignore_index=True)

In [ ]:
temp2022 = temp2022[['GEOID', 'geometry']]
temp2022['year'] = 2020

temp2020 = temp2020[['GEOID', 'geometry']]
temp2020['year'] = 2020

temp2019 = temp2019[['GEOID', 'geometry']]
temp2019['year'] = 2010

temp2018 = temp2018[['GEOID', 'geometry']]
temp2018['year'] = 2010

temp2017 = temp2017[['GEOID', 'geometry']]
temp2017['year'] = 2010

temp2016 = temp2016[['GEOID', 'geometry']]
temp2016['year'] = 2010

temp2015 = temp2015[['GEOID', 'geometry']]
temp2015['year'] = 2010

temp2014 = temp2014[['GEOID', 'geometry']]
temp2014['year'] = 2010

temp2013 = temp2013[['GEOID', 'geometry']]
temp2013['year'] = 2010

temp2012 = temp2012[['GEOID', 'geometry']]
temp2012['year'] = 2010

temp2011 = temp2011[['GEOID', 'geometry']]
temp2011['year'] = 2010

temp2010 = temp2010[['GEOID10', 'geometry']]
temp2010['year'] = 2010

temp2010 = temp2010.rename(columns={"GEOID10": "GEOID"})

temp2000 = temp2000[['BKGPIDFP00', 'geometry']]
temp2000['year'] = 2000

temp2000 = temp2000.rename(columns={"BKGPIDFP00": "GEOID"})

temp1990['TRACTSUF'].replace(np.nan, '000', inplace=True)

# Create 'GEOID' column by concatenating 'ST', 'CO', 'TRACTBASE', and 'TRACTSUF'
temp1990['GEOID'] = temp1990['ST'] + temp1990['CO'] + temp1990['TRACTBASE'] + temp1990['TRACTSUF']

temp1990 = temp1990[['GEOID', 'geometry']]
temp1990['year'] = 1990

In [ ]:
list_censusBG_2 = []

In [ ]:
list_censusBG = temp1990['GEOID'].dropna().drop_duplicates().astype(str).tolist()

# add preceding 0 to make 4-digit zip codes into 5-digit
list_censusBG = [censusBG.zfill(12) for censusBG in list_censusBG]

In [ ]:
len(list_censusBG)

In [ ]:
list_censusBG_2 = list_censusBG_2 + list_censusBG

In [ ]:
len(list_censusBG_2)

In [ ]:
list_censusBG_2 = list(set(list_censusBG_2))

In [ ]:
len(list_censusBG_2)

In [ ]:
#CensusBG list for NC

list_censusBG = df['censusBlockGroupFips'].dropna().drop_duplicates().tolist()

list_censusBG = [str(int(float(i))) for i in list_censusBG]

# add preceding 0 to make 4-digit zip codes into 5-digit
list_censusBG = [censusBG.zfill(12) for censusBG in list_censusBG]

In [ ]:
# Convert both lists to sets
set_bg = set(list_censusBG)
set_bg_2 = set(list_censusBG_2)

# Find the intersection of the two sets
common_bg = set_bg.intersection(set_bg_2)

# Print the number of common zip codes
print("Number of common BG:", len(common_bg))

In [ ]:
len(common_bg)/len(list_censusBG)

In [ ]:
len(common_bg)

In [ ]:
Tract_list_df2022 = temp2022[temp2022['GEOID'].isin(common_bg)]
Tract_list_df2020 = temp2020[temp2020['GEOID'].isin(common_bg)]
Tract_list_df2019 = temp2019[temp2019['GEOID'].isin(common_bg)]
Tract_list_df2018 = temp2018[temp2018['GEOID'].isin(common_bg)]
Tract_list_df2017 = temp2017[temp2017['GEOID'].isin(common_bg)]
Tract_list_df2016 = temp2016[temp2016['GEOID'].isin(common_bg)]
Tract_list_df2015 = temp2015[temp2015['GEOID'].isin(common_bg)]
Tract_list_df2014 = temp2014[temp2014['GEOID'].isin(common_bg)]
Tract_list_df2013 = temp2013[temp2013['GEOID'].isin(common_bg)]
Tract_list_df2012 = temp2012[temp2012['GEOID'].isin(common_bg)]
Tract_list_df2011 = temp2011[temp2011['GEOID'].isin(common_bg)]
Tract_list_df2010 = temp2010[temp2010['GEOID'].isin(common_bg)]
Tract_list_df2000 = temp2000[temp2000['GEOID'].isin(common_bg)]
Tract_list_df1990 = temp1990[temp1990['GEOID'].isin(common_bg)]

In [ ]:
Tract_df = pd.concat([Tract_list_df2022, Tract_list_df2020, Tract_list_df2019,Tract_list_df2013,Tract_list_df2012, Tract_list_df2011, Tract_list_df2010, Tract_list_df2000, Tract_list_df1990], ignore_index=True)

In [ ]:
Tract_df2 = Tract_df.drop_duplicates()

In [ ]:
Tract_df.shape[0] 

In [ ]:
Tract_df2.shape[0] 

In [ ]:
Tract_df2 = Tract_df2.drop_duplicates(subset=['GEOID', 'year'])

In [ ]:
Tract_df2.shape[0] 

In [ ]:
chunk_size = 40000  # adjust based on your system's capabilities
chunks = [x for x in range(0, len(Tract_df2), chunk_size)]

for start in chunks:
    end = start + chunk_size
    temp_df = Tract_df2.iloc[start:end].copy()
    temp_df['geometry'] = temp_df['geometry'].apply(lambda geom: geom.wkt)
    temp_df.to_parquet(f"Tract_geometry_{start}_{end}.parquet.gzip", compression='gzip')